# Chatper 3 exercise 1 & 2

#### 1. Try to build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set. Hint: the KNeighborsClassifier works quite well for this task; you just need to find good hyperparameter values (try a grid search on the weights and n_neighbors hyperparameters).

In [4]:
import numpy as np
from scipy.ndimage import shift
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
mnist = fetch_openml("mnist_784", as_frame=False, parser="auto")

data_train, data_test = mnist.data[:60000], mnist.data[60000:]
labels_train, labels_test = mnist.target[:60000], mnist.target[60000:]

In [2]:
knn = KNeighborsClassifier(n_neighbors=5, weights="uniform")
cross_val_score(
    knn, data_train, labels_train, cv=3, scoring="accuracy", n_jobs=4, verbose=1
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   43.8s finished


array([0.9676 , 0.9671 , 0.96755])

Basically there already :)

In [4]:
params = {
    "n_neighbors": range(3, 8),
    "weights": ["uniform", "distance"],
}

classifier = GridSearchCV(
    estimator=knn, param_grid=params, scoring="accuracy", n_jobs=4, cv=3, verbose=3
)
classifier.fit(data_train, labels_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[CV 1/3] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  36.2s
[CV 1/3] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  37.5s
[CV 3/3] END ....n_neighbors=3, weights=uniform;, score=0.968 total time=  39.7s
[CV 2/3] END ....n_neighbors=3, weights=uniform;, score=0.968 total time=  39.7s
[CV 2/3] END ...n_neighbors=3, weights=distance;, score=0.969 total time=  34.6s
[CV 3/3] END ...n_neighbors=3, weights=distance;, score=0.969 total time=  33.7s
[CV 1/3] END ....n_neighbors=4, weights=uniform;, score=0.966 total time=  37.3s
[CV 2/3] END ....n_neighbors=4, weights=uniform;, score=0.966 total time=  39.0s
[CV 1/3] END ...n_neighbors=4, weights=distance;, score=0.971 total time=  36.3s
[CV 3/3] END ....n_neighbors=4, weights=uniform;, score=0.967 total time=  37.8s
[CV 2/3] END ...n_neighbors=4, weights=distance;, score=0.970 total time=  35.0s
[CV 3/3] END ...n_neighbors=4, weights=distance;, score=0.970 total time=  35.1s
[CV 1/3] END ....n_neighbors

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': range(3, 8),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=3)

In [5]:
classifier.best_params_  # {'n_neighbors': 4, 'weights': 'distance'}

{'n_neighbors': 4, 'weights': 'distance'}

In [5]:
best_knn = KNeighborsClassifier(n_neighbors=4, weights="distance")
cross_val_score(
    best_knn, data_train, labels_train, scoring="accuracy", n_jobs=4, verbose=1
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  3.6min finished


array([0.973     , 0.97225   , 0.96991667, 0.97091667, 0.972     ])

All splits above or almost at 97%

In [9]:
best_knn.fit(data_train, labels_train)
accuracy_score(best_knn.predict(data_test), labels_test)

0.9714

#### 2. Write a function that can shift an MNIST image in any direction (left, right, up, or down) by one pixel.⁠ Then, for each image in the training set, create four shifted copies (one per direction) and add them to the training set. Finally, train your best model on this expanded training set and measure its accuracy on the test set. You should observe that your model performs even better now! This technique of artificially growing the training set is called data augmentation or training set expansion.

In [10]:
def shift_image(image, direction):
    if direction == "left":
        return shift(image.reshape(28, 28), (0, -1), cval=0).flatten()
    elif direction == "right":
        return shift(image.reshape(28, 28), (0, 1), cval=0).flatten()
    elif direction == "up":
        return shift(image.reshape(28, 28), (-1, 0), cval=0).flatten()
    elif direction == "down":
        return shift(image.reshape(28, 28), (1, 0), cval=0).flatten()
    else:
        return "Invalid direction!"

In [11]:
shifted_images = []
shifted_labels = []

for image, label in zip(data_train, labels_train):
    for direction in ["left", "right", "up", "down"]:
        shifted_images.append(shift_image(image, direction))
        shifted_labels.append(label)

In [12]:
shifted_images = np.array(shifted_images)
shifted_labels = np.array(shifted_labels)

In [13]:
data_train_augmented = np.concatenate((data_train, shifted_images))
labels_train_augmented = np.concatenate((labels_train, shifted_labels))
data_train_augmented.shape

(300000, 784)

In [14]:
best_knn = KNeighborsClassifier(n_neighbors=4, weights="distance")
best_knn.fit(data_train_augmented, labels_train_augmented)
accuracy_score(best_knn.predict(data_test), labels_test)

0.9763